In [3]:
# import modules
import os
import requests
import urllib3
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import folium
import polyline
import base64
from tqdm import tqdm
import json

# disable warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [17]:
data = open("data.json","r")
# normalize data
activities = json.loads(data.read())
activities = pd.json_normalize(activities)
#df= pd.DataFrame(activities)

In [10]:
df.dtypes

resource_state                     int64
athlete                           object
name                              object
distance                         float64
moving_time                        int64
elapsed_time                       int64
total_elevation_gain             float64
type                              object
sport_type                        object
workout_type                     float64
id                                 int64
start_date                        object
start_date_local                  object
timezone                          object
utc_offset                       float64
location_city                     object
location_state                    object
location_country                  object
achievement_count                  int64
kudos_count                        int64
comment_count                      int64
athlete_count                      int64
photo_count                        int64
map                               object
trainer         

In [20]:
# add decoded summary polylines
activities['map.polyline'] = activities['map.summary_polyline'].apply(polyline.decode)

In [21]:
# convert data types
activities.loc[:, 'start_date'] = pd.to_datetime(activities['start_date']).dt.tz_localize(None)
activities.loc[:, 'start_date_local'] = pd.to_datetime(activities['start_date_local']).dt.tz_localize(None)
# convert values
activities.loc[:, 'distance'] /= 1000 # convert from m to km
activities.loc[:, 'average_speed'] *= 3.6 # convert from m/s to km/h
activities.loc[:, 'max_speed'] *= 3.6 # convert from m/s to km/h
# set index
activities.set_index('start_date_local', inplace=True)
# drop columns
activities.drop(
    [
        'map.summary_polyline', 
        'resource_state',
        'external_id', 
        'upload_id', 
        'location_city', 
        'location_state', 
        'has_kudoed', 
        'start_date', 
        'athlete.resource_state', 
        'utc_offset', 
        'map.resource_state', 
        'athlete.id', 
        'visibility', 
        'heartrate_opt_out', 
        'upload_id_str', 
        'from_accepted_tag', 
        'map.id', 
        'manual', 
        'private', 
        'flagged', 
    ], 
    axis=1, 
    inplace=True
)

In [40]:
# select one activity
my_ride = activities.iloc[0, :] # first activity (most recent)
# plot ride on map
centroid = [
    np.mean([coord[0] for coord in my_ride['map.polyline']]), 
    np.mean([coord[1] for coord in my_ride['map.polyline']])
]
m = folium.Map(location=centroid, zoom_start=10)
folium.PolyLine(my_ride['map.polyline'], color='red').add_to(m)
display(m)